# Avalanche Danger Level Forecast: Preliminary Modeling

I am going to plug out newly created model-ready dataset into a few out-of-the-box models

## Import Tools

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# sklearn 
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler

# pandas
from pandas.plotting import scatter_matrix

## Import Data

In [2]:
avi = pd.read_csv('SnowWeatherCleanROS.csv')
avi.head()

,Unnamed: 0,avi_danger,avg_wind,temp_max_swing,temp_max_swing_from_avg,year,month,day,temp_max,temp_min,...,prevailing_wind_E_2,prevailing_wind_N_2,prevailing_wind_NE_2,prevailing_wind_NW_2,prevailing_wind_S_2,prevailing_wind_SE_2,prevailing_wind_SW_2,prevailing_wind_W_2,three_day_snow_2,next_day_avi_danger
0,0,1.0,20.58,0.0,0.0,2010.0,12.0,20.0,15,5,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.2,3.0
1,1,3.0,35.12,3.0,0.0,2010.0,12.0,21.0,18,10,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.3,2.0
2,2,2.0,33.78,-3.0,0.0,2010.0,12.0,22.0,15,7,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,3.0
3,3,3.0,31.32,0.0,0.0,2010.0,12.0,23.0,15,6,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,4.3,2.0
4,4,2.0,32.44,2.0,1.4,2010.0,12.0,24.0,17,9,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,8.3,2.0


## Dataset at a Glance

In [3]:
avi.describe()

,Unnamed: 0,avi_danger,avg_wind,temp_max_swing,temp_max_swing_from_avg,year,month,day,temp_max,temp_min,...,prevailing_wind_E_2,prevailing_wind_N_2,prevailing_wind_NE_2,prevailing_wind_NW_2,prevailing_wind_S_2,prevailing_wind_SE_2,prevailing_wind_SW_2,prevailing_wind_W_2,three_day_snow_2,next_day_avi_danger
count,1892.000000,1892.000000,1892.000000,1892.000000,1892.000000,1892.000000,1892.000000,1892.000000,1892.000000,1892.000000,...,1892.000000,1892.000000,1892.000000,1892.000000,1892.000000,1892.000000,1892.000000,1892.000000,1892.000000,1892.00000
mean,945.500000,2.213531,41.569268,0.438161,0.578330,1989.741543,3.602537,16.549683,19.517970,3.720930,...,0.066068,0.108879,0.067125,0.323467,0.110465,0.071882,0.116808,0.528541,4.607981,2.50000
std,546.317673,0.910522,16.356370,10.674199,12.429033,225.160208,3.252230,8.475751,13.234699,14.170538,...,0.462640,0.499376,0.463630,0.609305,0.500615,0.468028,0.505488,0.633733,5.034305,1.11833
min,0.000000,1.000000,4.000000,-44.000000,-53.800000,4.000000,1.000000,1.000000,-26.000000,-40.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.00000
25%,472.750000,1.000000,29.530000,-5.000000,-7.400000,2012.000000,2.000000,10.000000,11.000000,-6.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.100000,1.75000
50%,945.500000,2.000000,40.490000,2.000000,1.000000,2015.000000,3.000000,17.000000,19.000000,5.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,2.50000
75%,1418.250000,3.000000,52.847500,7.000000,9.400000,2018.000000,4.000000,24.000000,28.000000,12.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,6.300000,3.25000
max,1891.000000,4.000000,110.060000,39.000000,34.600000,2020.000000,12.000000,31.000000,57.000000,47.000000,...,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,34.200000,4.00000


In [4]:
avi.columns

Index(['Unnamed: 0', 'avi_danger', 'avg_wind', 'temp_max_swing',
       'temp_max_swing_from_avg', 'year', 'month', 'day', 'temp_max',
       'temp_min', 'water_equivalent', 'snow_fall', 'snow_depth_6am',
       'wind_speed_sum', 'sunshine_percent', 'west_wind_hours',
       'northwest_wind_hours', 'prevailing_wind_E', 'prevailing_wind_N',
       'prevailing_wind_NE', 'prevailing_wind_NW', 'prevailing_wind_S',
       'prevailing_wind_SE', 'prevailing_wind_SW', 'prevailing_wind_W',
       'prevailing_wind_na', 'three_day_snow', 'five_day_snow', 'avi_danger_1',
       'avg_wind_1', 'temp_max_swing_1', 'temp_max_swing_from_avg_1',
       'temp_max_1', 'temp_min_1', 'water_equivalent_1', 'snow_fall_1',
       'snow_depth_6am_1', 'wind_speed_sum_1', 'sunshine_percent_1',
       'west_wind_hours_1', 'northwest_wind_hours_1', 'prevailing_wind_E_1',
       'prevailing_wind_N_1', 'prevailing_wind_NE_1', 'prevailing_wind_NW_1',
       'prevailing_wind_S_1', 'prevailing_wind_SE_1', 'prevailing_wi

In [6]:
avi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1892 entries, 0 to 1891
Data columns (total 74 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Unnamed: 0                 1892 non-null   int64  
 1   avi_danger                 1892 non-null   float64
 2   avg_wind                   1892 non-null   float64
 3   temp_max_swing             1892 non-null   float64
 4   temp_max_swing_from_avg    1892 non-null   float64
 5   year                       1892 non-null   float64
 6   month                      1892 non-null   float64
 7   day                        1892 non-null   float64
 8   temp_max                   1892 non-null   int64  
 9   temp_min                   1892 non-null   int64  
 10  water_equivalent           1892 non-null   float64
 11  snow_fall                  1892 non-null   float64
 12  snow_depth_6am             1892 non-null   float64
 13  wind_speed_sum             1892 non-null   int64

## Filter Dataset

In [7]:
avi = avi.drop(['Unnamed: 0'], axis=1)

In [8]:
avi = avi[avi['avi_danger'].notnull()]
avi.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1892 entries, 0 to 1891
Data columns (total 73 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   avi_danger                 1892 non-null   float64
 1   avg_wind                   1892 non-null   float64
 2   temp_max_swing             1892 non-null   float64
 3   temp_max_swing_from_avg    1892 non-null   float64
 4   year                       1892 non-null   float64
 5   month                      1892 non-null   float64
 6   day                        1892 non-null   float64
 7   temp_max                   1892 non-null   int64  
 8   temp_min                   1892 non-null   int64  
 9   water_equivalent           1892 non-null   float64
 10  snow_fall                  1892 non-null   float64
 11  snow_depth_6am             1892 non-null   float64
 12  wind_speed_sum             1892 non-null   int64  
 13  sunshine_percent           1892 non-null   int64

## Fill Remaing NA's

In [9]:
avi = avi.fillna(0)
avi[avi.avi_danger == 5] = 4
avi[avi.next_day_avi_danger == 5] = 4
avi.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1892 entries, 0 to 1891
Data columns (total 73 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   avi_danger                 1892 non-null   float64
 1   avg_wind                   1892 non-null   float64
 2   temp_max_swing             1892 non-null   float64
 3   temp_max_swing_from_avg    1892 non-null   float64
 4   year                       1892 non-null   float64
 5   month                      1892 non-null   float64
 6   day                        1892 non-null   float64
 7   temp_max                   1892 non-null   int64  
 8   temp_min                   1892 non-null   int64  
 9   water_equivalent           1892 non-null   float64
 10  snow_fall                  1892 non-null   float64
 11  snow_depth_6am             1892 non-null   float64
 12  wind_speed_sum             1892 non-null   int64  
 13  sunshine_percent           1892 non-null   int64

### Standardizing the Data

In [10]:
#standardizer 
def standardize(X_train, X_test):
    scaler = StandardScaler()
    # Fitting and transforming training data
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    # Tranforming testing data based on traning fit (prevent data leakage)
    X_test = scaler.transform(X_test)
    return X_train, X_test


## Splitting Dataset

In [11]:
x = avi.iloc[:, 0:avi.shape[1]-2]
y = avi.iloc[:, avi.shape[1]-1]

print(x.shape)
print(y.shape)


(1892, 71)
(1892,)


In [12]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

In [13]:
# Scaling
x_train, x_test = standardize(x_train, x_test)

# Logistic Regression

In [26]:

penalty = ['l1', 'l2', 'elasticnet']
tol = [0.0001, 0.001, 0.01, 0.1]
C = [0.001, 0.01, 0.1, 1, 10, 100, 1000] 
solver = ['liblinear', 'newton-cg', 'lbfgs', 'sag', 'saga']
param_distributions = dict(penalty=penalty,
                           tol=tol,
                           C=C,
                           solver = solver)


lr = LogisticRegression(max_iter = 10000)
lr_cv = RandomizedSearchCV(estimator=lr, param_distributions= param_distributions, n_iter=5, scoring='f1_weighted')
lr_cv.fit(x_train, y_train)
y_pred_lr = lr_cv.predict(x_test)
print(accuracy_score(y_test, y_pred_lr))
print(cross_val_score(lr, x_train, y_train, cv=3))
print(lr_cv.best_params_)

/Users/lowell/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/lowell/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/lowell/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 1304, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/Users/lowell/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 442, in _check_solver
    raise ValueError("Solver %s supports only 'l2' or 'none' penalties, "
ValueError: Solver newton-cg supports only 'l2' or 'none' penalties, got l1 penalty.

  warnings.warn("Estimator fit failed. The score on this train-test"
/Users/lowell

0.5898520084566596
[0.61522199 0.59196617 0.63002114]
{'tol': 0.0001, 'solver': 'newton-cg', 'penalty': 'l2', 'C': 100}


In [27]:
print(confusion_matrix(y_test, y_pred_lr))

[[85 21  7  3]
 [17 52 26 12]
 [10 38 59 32]
 [ 3 15 10 83]]


In [28]:
print(classification_report(y_test, y_pred_lr))

              precision    recall  f1-score   support

         1.0       0.74      0.73      0.74       116
         2.0       0.41      0.49      0.45       107
         3.0       0.58      0.42      0.49       139
         4.0       0.64      0.75      0.69       111

    accuracy                           0.59       473
   macro avg       0.59      0.60      0.59       473
weighted avg       0.59      0.59      0.59       473



# Support Vector Machine (SVM)

In [32]:
param_distributions = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['rbf']}  


svm = SVC()
svm_cv = RandomizedSearchCV(estimator=svm, param_distributions= param_distributions, n_iter=5, scoring='f1_weighted')
svm_cv.fit(x_train, y_train)
y_pred_svm = svm_cv.predict(x_test)
print(accuracy_score(y_test, y_pred_svm))
print(cross_val_score(svm, x_train, y_train, cv=3))
print(svm_cv.best_params_)

0.7463002114164905
[0.68287526 0.62156448 0.67653277]
{'kernel': 'rbf', 'gamma': 0.01, 'C': 1000}


In [33]:
print(confusion_matrix(y_test, y_pred_svm))

[[ 90  20   6   0]
 [ 24  53  26   4]
 [  7  26  99   7]
 [  0   0   0 111]]


In [34]:
print(classification_report(y_test, y_pred_svm))

              precision    recall  f1-score   support

         1.0       0.74      0.78      0.76       116
         2.0       0.54      0.50      0.51       107
         3.0       0.76      0.71      0.73       139
         4.0       0.91      1.00      0.95       111

    accuracy                           0.75       473
   macro avg       0.74      0.75      0.74       473
weighted avg       0.74      0.75      0.74       473



# K-Nearest Neighbors

In [35]:
#List Hyperparameters that we want to tune.
leaf_size = list(range(1,50))
n_neighbors = list(range(1,30))
p=[1,2]
#Convert to dictionary
param_distributions = dict(leaf_size=leaf_size, n_neighbors=n_neighbors, p=p)

knn = KNeighborsClassifier()
knn_cv = RandomizedSearchCV(estimator=knn, param_distributions= param_distributions, n_iter=5, scoring='f1_weighted')
knn_cv.fit(x_train, y_train)
y_pred_knn = knn_cv.predict(x_test)
print(accuracy_score(y_test, y_pred_knn))
print(cross_val_score(knn, x_train, y_train, cv=3))
print(knn_cv.best_params_)

0.642706131078224
[0.58350951 0.54968288 0.56871036]
{'p': 2, 'n_neighbors': 3, 'leaf_size': 46}


In [36]:
print(confusion_matrix(y_test, y_pred_knn))

[[ 86  16  10   4]
 [ 29  36  32  10]
 [ 26  25  71  17]
 [  0   0   0 111]]


In [37]:
print(classification_report(y_test, y_pred_knn))

              precision    recall  f1-score   support

         1.0       0.61      0.74      0.67       116
         2.0       0.47      0.34      0.39       107
         3.0       0.63      0.51      0.56       139
         4.0       0.78      1.00      0.88       111

    accuracy                           0.64       473
   macro avg       0.62      0.65      0.63       473
weighted avg       0.62      0.64      0.62       473



# Gradient Boosting Classifier

In [ ]:
param_distributions = {'learning_rate':[0.15,0.1,0.05,0.01,0.005,0.001], 
                       'n_estimators':[100,250,500,750,1000,1250,1500,1750],
                       'max_depth':[2,3,4,5,6,7] }

gbc = GradientBoostingClassifier()
gbc_cv = RandomizedSearchCV(estimator=gbc, param_distributions= param_distributions, n_iter=5, scoring='f1_weighted')
gbc_cv.fit(x_train, y_train)
y_pred_gbc = gbc_cv.predict(x_test)
print(accuracy_score(y_test, y_pred_gbc))
print(cross_val_score(gbc, x_train, y_train, cv=3))
print(gbc_cv.best_params_)

In [67]:
print(confusion_matrix(y_test, y_pred_gbc))

[[76 15  3  1]
 [ 9 73 23  3]
 [10 38 39  1]
 [ 1  7 13  1]]


In [68]:
print(classification_report(y_test, y_pred_knn))

              precision    recall  f1-score   support

         1.0       0.71      0.74      0.72        95
         2.0       0.51      0.70      0.59       108
         3.0       0.52      0.39      0.44        88
         4.0       0.00      0.00      0.00        22

    accuracy                           0.58       313
   macro avg       0.44      0.46      0.44       313
weighted avg       0.54      0.58      0.55       313



/Users/lowell/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
